<a href="https://colab.research.google.com/github/michalisgalanis/garbage-detection-internship/blob/main/Garbage_Detection_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A. Pre-processing**

## Acquiring the TACO dataset
TACO is a growing image dataset of waste in the wild. It contains images of litter taken under diverse environments: woods, roads and beaches. These images are manually labeled and segmented according to a hierarchical taxonomy to train and evaluate object detection algorithms. Currently, images are hosted on
Flickr and we have a server that is collecting more images and annotations.

Get more information [here](https://github.com/pedropro/TACO).

In [ ]:
# 0. Clear Data
import os, shutil
if os.path.exists('sample_data/'): shutil.rmtree('sample_data/')

In [ ]:
# 2.1 Clone & Download dataset
!git clone https://github.com/pedropro/TACO
%cd /content/TACO
!python3 download.py

In [ ]:
# 2.2 OR Upload TACO.zip with 2-3 images (for simplicity's sake)
import os, shutil

%cd /content
if os.path.exists('/content/TACO'): shutil.rmtree('/content/TACO')
!unzip /content/TACO.zip
if os.path.exists('/content/TACO.zip'): os.remove('TACO.zip')

## Organize files & folders

In [ ]:
# 3. Move all images from multiple batch folders to a new single folder.
import os
import shutil

root_path = 'TACO'
data_path = os.path.join(root_path, 'data')
image_path = os.path.join(data_path, 'images')

# Create new folder 'images'
if os.path.exists(image_path):
    shutil.rmtree(image_path)
os.mkdir(image_path)

for batch in os.listdir(data_path):
    if os.path.isdir(data_path) and 'batch' in batch:
        for image in os.listdir(os.path.join(data_path, batch)):
            shutil.move(os.path.join(data_path, batch, image), os.path.join(image_path, batch + '_' + image))
        shutil.rmtree(os.path.join(data_path, batch))

In [ ]:
# 4. Split dataset into train and val
import random
import math

train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')
json_filename = 'annotations.json'
train_val_ratio = 0.9

images = os.listdir(image_path)
random.shuffle(images)
train_images = images[0 : math.floor(train_val_ratio * len(images))]
val_images = images[math.floor(train_val_ratio * len(images)):]

# Create train & val folders
if os.path.exists(train_path):
    shutil.rmtree(train_path)
os.mkdir(train_path)

if os.path.exists(val_path):
    shutil.rmtree(val_path)
os.mkdir(val_path)

for train_image in train_images:
    shutil.move(os.path.join(image_path, train_image), os.path.join(train_path, train_image))

for val_image in val_images:
    shutil.move(os.path.join(image_path, val_image), os.path.join(val_path, val_image))

shutil.rmtree(image_path)

# Copy annotations to both splits
shutil.copy(os.path.join(data_path, json_filename), os.path.join(train_path, json_filename))
shutil.copy(os.path.join(data_path, json_filename), os.path.join(val_path, json_filename))
os.remove(os.path.join(data_path, json_filename))

# **B. Training**

### Setting up training process

In [ ]:
# 1. Clone Mask_RCNN repo
import os, shutil

model_path = '/content/Mask_RCNN'
if os.path.exists(model_path): shutil.rmtree(model_path)

!git clone https://github.com/matterport/Mask_RCNN /content/Mask_RCNN

[Errno 2] No such file or directory: 'Mask_RCNN/'
/content
Cloning into '/content/Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.85 MiB | 40.34 MiB/s, done.
Resolving deltas: 100% (563/563), done.


In [ ]:
# 2. Configure Dataset
import sys
import os
import json
import skimage.io


%cd /content/Mask_RCNN

# Root directory of the project
ROOT_DIR = os.path.abspath("/TACO") 
sys.path.append(ROOT_DIR)

train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')
json_filename = 'annotations.json'

from Mask_RCNN.mrcnn.config import Config
from Mask_RCNN.mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

class GarbageConfig(Config):
    # Give the configuration a recognizable name
    NAME = "garbage"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 60  # Background + garbage categories

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

class GarbageDataset(utils.Dataset):

    def load_garbage(self, dataset_dir, subset):
        # Read JSON file
        json_dict = json.load(open(os.path.join(train_path, json_filename)))
        for category in json_dict['categories']:
            # Add every class (60 categories)
            self.add_class(category['supercategory'], category['id'], category['name'])

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        images_dict = json_dict['images']
        annotations = json_dict['annotations']

        for image in images_dict:
            if (image['file_name'].rpartition('/')[0] + '_' + image['file_name'].rpartition('/')[2]) in os.listdir(dataset_dir):
                print('Image',image['file_name'], 'found!')
                polygons = []
                for ann in annotations:
                    if ann['image_id'] == image['id']:
                        polygons.append(ann['segmentation'][0])

                image_path = os.path.join(dataset_dir, (image['file_name'].rpartition('/')[0] + '_' + image['file_name'].rpartition('/')[2]))
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

                print(image_path, height, width)
            #else:
            #    print('Image',image['file_name'], 'not found!')

            self.add_image(
                "garbage",
                image_id = image['file_name'].rpartition('/'),  # use file name as a unique image id
                path = image_path,
                width = width, height = height,
                polygons = polygons)

    def train(model):
        # Training dataset.
        dataset_train = GarbageDataset()
        dataset_train.load_garbage(args.dataset, "train")
        dataset_train.prepare()

        # Validation dataset
        dataset_val = GarbageDataset()
        dataset_val.load_garbage(args.dataset, "val")
        dataset_val.prepare()

        print("Training network heads")
        model.train(dataset_train, dataset_val,
                    learning_rate=config.LEARNING_RATE,
                    epochs=30,
                    layers='heads')       

/content/Mask_RCNN


In [ ]:
# TODO
def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "balloon":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

def image_reference(self, image_id):
    """Return the path of the image."""
    info = self.image_info[image_id]
    if info["source"] == "balloon":
        return info["path"]
    else:
        super(self.__class__, self).image_reference(image_id)

In [ ]:
%cd /content/
!ls

config = GarbageConfig()
#config.display()


folders = ['train', 'val']
for folder in folders:
    for image in os.listdir(os.path.join(ROOT_DIR, 'data', folder)):
        # Resize Images
        image = skimage.io.imread(os.path.join(ROOT_DIR, 'data', folder, image))

        original_shape = image.shape
        image, window, scale, padding, _ = utils.resize_image(
            image, 
            min_dim=config.IMAGE_MIN_DIM, 
            max_dim=config.IMAGE_MAX_DIM,
            mode=config.IMAGE_RESIZE_MODE)
        skimage.io.imsave(os.path.join(ROOT_DIR, 'data', folder), image)

model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
# Download weights file
if not os.path.exists(weights_path):
    utils.download_trained_weights(weights_path)


print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
train(model)

### Train


# **C. Testing**